# Super Lucky Palindromes Notebook

Original problem : https://www.spoj.com/problems/CTPLUCKY/

This notebook will present my work on the problem I had really fun solving for the past three months.
At first, I solved this problem with a naive algorithm (v1 and v2) but they were not fast enough to solve the 10^18th index that problem's statement wants (reaching this point would have taken a year with v2 I think).

In this notebook you'll find the predata code which helps to get initial parameters for v1, v2 and to better understand the fundamentals off next algorithms. <br>
All five algorithm's versions I did to solve the problem. <br>
At the end a time comparison between these algorithms which will suprise you I think :)

Little vocabulary :
- A lucky number is a positive integer only made of "4" and "7"
- A super lucky number is a lucky number where its amount of digit and its amount of "4" or "7" is a lucky number
- A super lucky palindrome is a super lucky number which is also a [palindrome](https://en.wikipedia.org/wiki/Palindromic_number)
- Super Lucky Palindrome will be called SLP

Execute every cell step by step and enjoy the reading as well as the results. Don't hesitate to change the values as your own desires :D

## -=-=-=-=-=-=-=-=-

## Predata

This predata code computes initial parameters to make a slightly faster algorithm.<br>
These values are always the same, that's why I can store them in array.

This is how I get these values :<br>
First, I know we're looking for a palindrome. It means that I can work with only one side of the number and mirror my result at the end.<br>
Remember, a SLP must have specific amounts of "4" and "7". Within one part, I need to find what are all possible amount that can create a valid SLP. I called these values "pair numbers". (Pair because if you know the amount of "4" you can guess the amount of "7" by doing length-"4")<br>
Once I do that, I can use [binomial coeficient](https://en.wikipedia.org/wiki/Combination) to directly know how much possibilities exists with the length I am working on by using the pair numbers I just found.<br>

This code print 3 differents values :
- Possible numbers length which are 4, 7, 44, 47, 74, 77, 444. Going further is useless because the 10^18th index is contained in the 444 length.
- Pair numbers. Mind that pair numbers represent the amount for only one side of the palindrome
- Amount of possibilities for each length.

There is a little issue I need to solve about odd length palindrome number and their middle digit.<br>
For this, I create a condition like : If the middle digit is a "4" , does it make a SLP ? (same with a "7")<br>
Hopefully only one solution is valid, the middle digit is either a "4" or a "7", never both.

All this predata code can be verified easily if you start to find the first SLP, you'll see the datas are matching with the results :p

In [ ]:
import scipy.special

def is_lucky(number): #this return 1 if the number is a lucky number
    if any(x in str(number) for x in "01235689"):
        return 0
    return 1

def get_pair(length): #return pair numbers
    pair = []
    limit = length // 2
    is_odd = length % 2
    
    #About the big condition, it simulates every possible cases.
    #length-amount_4 is the amount of "7" and I always multiply by 2 to represent the full length of the final result
    #and +is_odd simulates the middle digit being a "4" or a "7"
    for amount_4 in range(0, limit+1): #I do +1 because there can be a "4" everywhere
        if is_lucky(amount_4*2) == 1 or is_lucky(length-amount_4*2) == 1 or \
        is_lucky(amount_4*2+is_odd) == 1 or is_lucky(length-amount_4*2-is_odd) == 1:
            pair.append(amount_4)
    return pair

def get_possibilities(length, pair): #return the amount of possibilities
    total = 0
    for i in pair:
        total += scipy.special.comb(length, i, exact=True) #get all possibilities. Exact=True return an integer
    return total

In [ ]:
print("length | pair numbers                                 | possibilities\n")
for i in range(4,445): #you can change the maximum for fun
    if is_lucky(i) == 1:
        pair = get_pair(i)
        possibilities = get_possibilities(i//2, pair)
        print("   %3d | %-44s | %d" % (i, pair, possibilities))

## v1 : Naive algorithm

The idea is simple, create every possibilities and check for every possibility if its a SLP.<br>

Let's explain how it works. First, I need to get the real query value.<br>
The real query symbolize the query we are looking for inside a given length. If you try the algorithm with queries 1, 3, 11, 475 you'll see that these solutions are filled with "4" only. So we need to get this real query value because all these solutions have the same real query value of 1.<br>

To get the real query it's quite simple, I substract to the initial query the values contained in range_list array which stores the amount of possibilities for each length. I do that until I reach a negative value. It's like saying "ok, my query will not be in this length, let's skip the possibilities of this length".

About the recursive function :<br>
Instead of having an empty array and placing either a "4" or a "7" inside it I create an all "4" (or "7") array and I just change the values if I have to. This way I can create a more generic code and a slightly faster algorithm.

In [12]:
import copy

class lucky_v1():
    def __init__(self): 
        self.length_list = [4, 7, 44, 47, 74, 77, 444]
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246514075237194788708873582383079842]
        self.pair_list = [[0,2], [0,1,2,3], [0,2,20,22], [0,1,2,3,20,21,22,23], [0,2,15,22,35,37], [0,1,2,3,15,16,22,23,35,36,37,38]]
        
        self.index = 0       #select the index of _list
        self.query = 0       #the real query
        self.limit = 0       #length // 2
        self.cursor = 1      #the amount of possibilities found. If equal to self.query then solution has been found
        self.solution = ""   #store solution
    
    def put(self, res, pos):
        res[pos] = "7"
        return res
    
    def generate(self, res, pos=0, num4=0, num7=0):
        if self.cursor > self.query: #once solution has been found we can end the recursive function
            return 
        if pos == self.limit: #we placed all possible '4' and '7'
            if num4 in self.pair_list[self.index] or num7 in self.pair_list[self.index]: #need to check if it's a SLP
                self.cursor += 1
                if self.cursor == self.query: #we hit the solution we're looking for
                    if self.length_list[self.index]%2 == 0: #if length is odd
                        self.solution = res + res[::-1]
                    else: #else we need to find what is the value of the middle digit
                        if num4*2 + 1 in self.length_list or num7*2 in self.length_list:
                            self.solution = res + ['4'] + res[::-1]
                        else:
                            self.solution = res + ['7'] + res[::-1]
        else:
            self.generate(res, pos + 1, num4 + 1, num7) #we simulate a placement of a "4"
            self.generate(self.put(copy.copy(res), pos), pos + 1, num4, num7 + 1) #we put a "7"
            
    def get_solution(self, query):
        self.query = query
        self.index = 0
        while self.query - self.range_list[self.index] > 0: #need to find what is the index of _list arrays
            self.query -= self.range_list[self.index]
            self.index += 1
        
        self.limit = self.length_list[self.index]//2
        self.cursor = 0
        
        self.generate(["4"]*self.limit) #generate() will create the solution inside the function
        
        return "".join(self.solution)

In [13]:
import time

def print_v1(query):
    v1 = lucky_v1()
    start = time.time()
    sol = v1.get_solution(query)
    timeE = time.time() - start
    print('%5d | %4d | %-47s | %5f' % (query, v1.query, sol, timeE))

print("query | real | solution                                        | time (s)")
for start, finish in zip([1, 240, 472], [21, 245, 482]):
    print()
    for query in range(start,finish):
        print_v1(query)

query | real | solution                                        | time (s)

    1 |    1 | 4444                                            | 0.000019
    2 |    2 | 7777                                            | 0.000008
    3 |    1 | 4444444                                         | 0.000009
    4 |    2 | 4477744                                         | 0.000009
    5 |    3 | 4747474                                         | 0.000008
    6 |    4 | 4774774                                         | 0.000011
    7 |    5 | 7447447                                         | 0.000011
    8 |    6 | 7474747                                         | 0.000012
    9 |    7 | 7744477                                         | 0.000012
   10 |    8 | 7777777                                         | 0.000012
   11 |    1 | 44444444444444444444444444444444444444444444    | 0.000035
   12 |    2 | 44444444444444444444777744444444444444444444    | 0.000031
   13 |    3 | 4444444444444444444747

As you can see above, it all depends about real query. The query 474 is way slower than 475. 

## -=-=-=-=-=-=-=-=-=-

## v2 : Naive algorithm - Optimized

Once the algorithm was working well patterns started to appear.<br>
I discovered that once you've found half of the solutions for a given length, you just have to change the "4" by "7" and the "7" by "4" to create the opposite solution. You can look at query 6 and 7, 5 and 8, 4 and 9 ... you'll see they are mirrored<br>
This way I can optimize the algorithm even more with the help of "self.reverse" which will exchange the "4" by "7".

The previous algorithm also has a big flaw because I'm computing every existing possibilities and then I have to check if the current result can become a SLP.<br>
For a 44 length number that we divide in two, there is 2^22 possibilities to check in total, which is 4194304.<br>
Inside these possibilities there is only 464 (check range_list) that can create a SLP. Which is 0.01%.

I created the function "is_possible" which can fix this issue by detecting as soon as possible if the current solution i'm computing will give a SLP or not.<br>
The idea is simple : With the current state of the result, does a solution still exists if I add a "7" ?<br>
Refering to the pair numbers, I know how much "4" or "7" I can have at the end.<br>
So if I don't have enough space left to write a correct amount of "7", it is useless to add a "7".

With this I'm sure that, once every digit is written, I do have an existing solution.<br>
I still have to compute the middle digit of odd length numbers tho.

In [ ]:
import copy

class lucky_v2():
    def __init__(self):
        self.length_list = [4, 7, 44, 47, 74, 77, 444]
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246514075237194788708873582383079842]
        self.pair_list = [[0,2], [0,1,2,3], [0,2,20,22], [0,1,2,3,20,21,22,23], [0,2,15,22,35,37], [0,1,2,3,15,16,22,23,35,36,37,38]]
        
        self.index = 0       #select the index of _list
        self.query = 0       #the real query
        self.limit = 0       #length // 2
        self.cursor = 1      #the amount of possibilities found. If equal to self.query then solution has been found
        self.reverse = False #the reverse feature
        self.solution = ""   #store solution
    
    def put(self, res, pos):
        res[pos] = "7"
        return res
    
    def is_possible(self, amount, space): #current amount ("4" or "7") and space is remaining digits to write
        idx = 0
        while self.pair_list[self.index][idx] <= amount: #we are going to place a digit, 
            idx += 1                                     #... so what is the first value above amount ?
        return self.pair_list[self.index][idx] <= amount + space #Can we reach this value with the space left ?
    
    def generate(self, res, pos=0, num4=0, num7=0):
        if self.cursor > self.query: #once a solution has been found we can end the recursive function
            return
        if pos == self.limit:
            self.cursor += 1
            if self.cursor == self.query:
                if self.length_list[self.index]%2 == 0: #if length is odd
                    self.solution = res + res[::-1]
                else: #else need to find what is the value of the middle digit
                    if num4*2 + 1 in self.length_list or num7*2 in self.length_list:
                        self.solution = res + ['4'] + res[::-1]
                    else:
                        self.solution = res + ['7'] + res[::-1]
                return
        else:
            if self.is_possible(num4, self.limit-pos) == True:
                self.generate(res, pos + 1, num4 + 1, num7)
            if self.is_possible(num7, self.limit-pos) == True:
                self.generate(self.put(copy.copy(res), pos), pos + 1, num4, num7 + 1)
    
    def get_solution(self, query):
        self.query = query
        self.index = 0
        while self.query - self.range_list[self.index] > 0: #need to find what is the index of _list arrays
            self.query -= self.range_list[self.index]
            self.index += 1
        
        self.reverse = False
        if self.query - (self.range_list[self.index]//2) > 0: #we apply the reverse feature
            self.query = self.range_list[self.index] - self.query + 1
            self.reverse = True
        
        self.limit = self.length_list[self.index]//2
        self.cursor = 0
        
        self.generate(["4"]*self.limit) #generate() will store the solution inside the function in self.solution
        
        if self.reverse == True: #reverse the solution if needed
            for i in range(self.length_list[self.index]):
                self.solution[i] = "4" if self.solution[i] == "7" else "7"
        self.solution = "".join(self.solution)
        return "".join(self.solution)

In [ ]:
import time

def print_v2(query):
    v2 = lucky_v2()
    start = time.time()
    sol = v2.get_solution(query)
    end = time.time() - start
    print('%7d | %7s | %-74s | %f' % (v2.query, v2.reverse, sol, end))

print("   real | reverse | solution                                                                   | time (s)")
for start, finish in zip([1, 240, 472, 400000], [16, 245, 482, 400005]):
    print()
    for query in range(start,finish):
        print_v2(query)

It seems that v1 and v2 takes almost the same amount of time to compute query 242 and 400000 respectively.<br>
Don't worry, it's just the beginning :)

## -=-=-=-=-=-=-=-=-=-

## v3 : Mathematical approach

Now the algorithms are going to change by a lot. Let's explain the new logic behind them.

We already use a part of this logic, in the predata code. There, I was able to find how much possibilities were contained inside a given length once pair numbers had been found.<br>
So by using the same logic, we're going to skip a good amount of solutions by putting "7" exactly where we have to.

Let's take the essence of the logic by using it on arbitrary parameters.<br> 
Exemple with these parameters : length = 4, pair = [0,2], query = 9. It means we are looking for the 9th solution within a length of 4 where we can only put either zero "7" OR two "7".
This gives these possibilities :<br>
- 1 > 4444
- 2 > 4477
- 3 > 4747
- 4 > 4774
- 5 > 7447
- 6 > 7474
- 7 > 7744

They are seven of them. The amount of these possibilities is calculated with the same algorithm in the predata code.<br>
BUT, my query is currently 9, so I am sure that the first "7" to put will NOT appear in this 4 length.

Now, If I calculate with a length of 5 it gives this :<br>
- 1 > 44444
- 2 > 44477
- 3 > 44747
- 4 > 44774
- 5 > 47447
- 6 > 47474
- 7 > 47744
- 8 > 74447
- 9 > 74474
- 10>74744
- 11>77444

The query 9 now appears inside these solution. I am definitely sure that the first "7" to put will be at this new index.<br>
So now I do a little trick, I decrease the value of query by the amount of possibilities from length-1. In others terms, I "remove" the 4 length possibilities (possibilities that doesn't have a "7" at their last position). Remember, there was 7 solutions in 4 length, so my query is now equal to 9-7 = 2.<br>
Mind that we also need to decrease the pair values because we just put one "7". That means the pair[0] won't be possible anymore !

And so we can continue with our current parameters : length = 1, pair = [1], query = 2. This gives :<br>
One solution in 1 length (74447). That is not enough solution, out query is 2. Let's check next length.<br>
Two solutions in 2 length (74447, 74474), our query is equal to 2 so we know that there is a "7" in this new solution.
And we're done because we put all the "7".

We are now going to do the same.<br>
1) We need to know the real query. To do that we have to :<br>
1.1) Try successives correct SLP length (4, 7, 44, 47, 74, 77, 444 ...)<br>
1.2) Get the pair numbers of this length<br>
1.2) Calculate the amount of possibilities this length have by using pair numbers<br>
1.3) Decrease the value of query by this amount<br>
1.4) If query is < 0, we got the length. Else we need to continue with the next possible length

2) Now we have the real query we will put "7" step by step<br>
2.1) Store all amount of possibilities for each value between 1 and length<br>
2.2) Detect what is the index of the maximum value <= real query<br>
2.3) Put one "7" in solution at this index<br>
2.4) Decrease real query by the amount found and decrease pair numbers by 1<br>
2.5) Update length by index<br>
2.6) Restart while length > 0 and real query > 0 and len(pair_number) > 0

There is some littles things i'm not talking about but here you have the essence of the algorithm.

I also removed the reverse feature because it was kinda useless, the algorithm was very fast already.

In [1]:
import scipy.special

class lucky_v3():
    def __init__(self):
        self.query = 0
        self.seven = 0
        self.solution = ""
        
    def is_lucky(self, number): #detect if number is a lucky number
        if any(x in str(number) for x in "01235689"):
            return 0
        return 1
    
    def get_length_from_index(self, index): #create correct SLP length
        binary = "{0:b}".format(index+2)
        res = ""
        for idx, i in enumerate(binary[1:]):
            res += "4" if i == "0" else "7"
        return int(res)
    
    def get_palindrome_pair(self, length):
        pair = []
        is_odd = length % 2
        limit = length // 2
        total = 0
        for x in range(0, limit+1):
            if self.is_lucky(x*2+is_odd) == 1 or self.is_lucky(x*2) == 1 or \
            self.is_lucky(length-x*2-is_odd) == 1 or self.is_lucky(length-x*2) == 1:
                pair.append(x)
        return pair
    
    def get_possibilities(self, length, pair): #return amount of possibilities
        total = 0
        for i in pair:
            total += scipy.special.comb(length, i, exact=True)
        return total
    
    def get_comb(self, length, pair): #return all possibilities by storing them in array
        comb = []
        for space in range(1, length+1):
            comb.append(self.get_possibilities(space, pair))
        return comb
    
    def generate(self, query, pair_numbers, length):
        pair = pair_numbers
        space = length
        self.solution = ["4"] * length
        self.seven = 0 #we assume that the solution is filled by "4" where some "4" will be replaced by "7"
        
        while query > 0:
            comb = self.get_comb(space, pair)
            idx = 0
            while query >= comb[idx]: #get the correct idx. query is always inferior to comb[-1], so we will never be out of range
                idx += 1
            self.solution[idx] = "7"
            self.seven += 1
            query -= comb[idx-1] #decrease the value of query by the amount of useless solutions
            space = idx
            for i in range(len(pair)): #update pair numbers
                pair[i] -= 1
            if pair[0] < 0: #delete because useless
                del pair[0]
        
        if query == 0: #here we put the rest of "7" that needs to be added
            for i in range(pair[0]):
                self.solution[i] = "7"
                self.seven += 1
        self.solution = "".join(self.solution)
        return self.solution
        
    def get_solution(self, query): #let's go
        self.query = query
        index = 0
        while self.query > 0: #we need to know the length of the solution
            length = self.get_length_from_index(index) #we first get the length of out current lucky number
            pair = self.get_palindrome_pair(length) #we determine the pair numbers of the length
            limit = length // 2
            total = self.get_possibilities(limit, pair) #we calculate how much possibilities exist
            self.query -= total
            index += 1
        self.query += total - 1       #need to go back one step to know what is the real query
        self.generate(self.query, pair, limit) #everything
        
        if length%2 == 0: #we create the whole number from one side of it
            self.solution = self.solution[::-1] + self.solution
        else:                                        #here we determine what is the value of the middle digit
            if self.is_lucky(self.seven*2 + 1) == 1 or self.is_lucky((limit - self.seven)*2) == 1:
                self.solution = self.solution[::-1] + "7" + self.solution
            else:
                self.solution = self.solution[::-1] + "4" + self.solution
        
        return self.solution

In [ ]:
import time

def print_v3(query):
    v3 = lucky_v3()
    start = time.time()
    sol = v3.get_solution(query)
    timeE = time.time() - start
    print("%19d | %f | %s (%d)" % (query, timeE, sol, len(sol)))

print("              query | time (s) | sol (length)")
for start, finish in zip([1, 240, 400000, 10**10, 10**15, 10**18, 10**100], [16, 245, 400005, 10**10+1, 10**15+1, 10**18+1, 10**100+1]):
    print()
    for query in range(start,finish):
        print_v3(query)

Here it is, solution 10^18 has been found in some miliseconds and 10^100 in less than a second<br>
Impressed ? We can go even further :D

## -=-=-=-=-=-=-=-=-=-=-

## v4 Mathematical approach - Opzimized

v4 is like v3 but with two little fixes.

The first issue is that the comb array is used to store all possibilities that exist between 1 and space.<br>
Remember that we want to get the first index where the amount of possibilities is higher than our current query. So instead of calculating everything, storing it and then searching it, we now just calculate what we need.<br>
Imagine that the number we are looking for is at the beginning of the array, all the remaining values have been calculated for nothing.

The second issue is that most of the time, the value of idx we need to get is at the end of the array. So, instead of starting at the beginning we start from the end.

The algorithm is almost the exact same, there is 5 line of code that changed. I have nothing much to say.

In [ ]:
import scipy.special

class lucky_v4():
    def __init__(self):
        self.query = 0
        self.seven = 0
        self.solution = ""
        
    def is_lucky(self, number): #detect if number is a lucky number
        if any(x in str(number) for x in "01235689"):
            return 0
        return 1
    
    def get_length_from_index(self, index): #create correct SLP length
        binary = "{0:b}".format(index+2)
        res = ""
        for idx, i in enumerate(binary[1:]):
            res += "4" if i == "0" else "7"
        return int(res)
    
    def get_palindrome_pair(self, length):
        pair = []
        is_odd = length % 2
        limit = length // 2
        total = 0
        for x in range(0, limit+1):
            if self.is_lucky(x*2+is_odd) == 1 or self.is_lucky(x*2) == 1 or \
            self.is_lucky(length-x*2-is_odd) == 1 or self.is_lucky(length-x*2) == 1:
                pair.append(x)
        return pair
    
    def get_possibilities(self, length, pair): #return the amount of possibilities
        total = 0
        for i in pair:
            total += scipy.special.comb(length, i, exact=True)
        return total
    
    def generate(self, query, pair_numbers, length):
        pair = pair_numbers
        space = length
        self.solution = ["4"] * length
        self.seven = 0 #we assume that the solution is filled by "4" where some of "4" will be replaced by "7"
        
        idx = space
        while query > 0:
            remove = query + 1
            while idx >= 0 and query < remove: #get the correct idx.
                remove = self.get_possibilities(idx, pair)
                idx -= 1
            self.solution[idx+1] = "7"
            self.seven += 1
            query -= remove #decrease the value of query by the amount of useless solutions
            space = idx
            for i in range(len(pair)): #update pair numbers
                pair[i] -= 1
            if pair[0] < 0: #delete because useless
                del pair[0]
        
        if query == 0: #here we put the rest of "7" that needs to be added
            for i in range(pair[0]):
                self.solution[i] = "7"
                self.seven += 1
        self.solution = "".join(self.solution)
        return self.solution
        
    def get_solution(self, query): #let's go
        self.query = query
        index = 0
        while self.query > 0: #we need to know the length of the solution
            length = self.get_length_from_index(index) #we first get the length of out current lucky number
            pair = self.get_palindrome_pair(length) #we determine the pair numbers of the length
            limit = length // 2
            total = self.get_possibilities(limit, pair) #we calculate how much possibilities exist
            self.query -= total
            index += 1
        self.query += total - 1       #go back one step to know what is the real query
        
        self.generate(self.query, pair, limit) #everything
        
        if length%2 == 0: #we create the whole number from one side of it
            self.solution = self.solution[::-1] + self.solution
        else:                                        #we determine what is the value of the middle digit
            if self.is_lucky(self.seven*2 + 1) == 1 or self.is_lucky((limit - self.seven)*2) == 1:
                self.solution = self.solution[::-1] + "7" + self.solution
            else:
                self.solution = self.solution[::-1] + "4" + self.solution
        return self.solution

In [ ]:
import time

def print_v4(query):
    v4 = lucky_v4()
    start = time.time()
    sol = v4.get_solution(query)
    timeE = time.time() - start
    print("%19d | %f | %s (%d)" % (query, timeE, sol, len(sol)))

print("              query | time (s) | sol (length)")
for start, finish in zip([1, 240, 380000, 10**10, 10**15, 10**18, 10**100], [16, 245, 380005, 10**10+1, 10**15+1, 10**18+1, 10**100+1]):
    print()
    for query in range(start,finish):
        print_v4(query)

Yeah. Much better. Take note that, the higher the length, the faster the algorithm gets between v3 and v4.

## -=-=-=-=-=-=-=-=-=-=-

## v5 Mathematical approach - The end

So here it is, the final version. <br>

What changed ? Two things.<br>
The first one is the verbose feature. You can now use lucky_v5(True) to enable it. That will displays some cool statistics during the computation of big length numbers. (10^100 and more)

Second is the apparition of the exponential search. Let's explain why it's here !<br>
What takes a huge amount of time is the self.get_possibilities(). If we can limit at maximum the amount of times this function is called we will optimize the algorithm even more.<br>
We know that the amount of possibilities between 0 and space is sorted and that the index we're looking for is mostly at the end of the array. We can use these clues to get the index we want with an exponential search.<br>
Even tho comb array is not used anymore, the logic behind the computation remain the same and we can apply this kind of search algorithm for our problem.

Reversed has been also implemented (it doesn't change much the result, it was just for the glory)<br>

Basically, everything has been implemented and I don't really see where I could optimize the algorithm even more.<br>
I'm really proud of this journey that comes to its end. It's been fun solving this.

Is there anything I want to do ? Not really. Thanks for reading.

In [1]:
import scipy.special
import time
from sys import stdout

class lucky_v5():
    def __init__(self, verbose=False):
        self.verbose = verbose
        self.start_time_global = 0
        self.start_time_segment = 0
        
        self.query = 0
        self.cursor = 0
        self.length = 0
        self.limit = 0
        self.pair = []
        self.seven_amount = 0
        self.reverse = False
        self.total = 0
        self.solution = ""
        
    def verbose_print(self, text_index): #ignore this, it's just display
        if self.verbose == False:
            return
                                        #stdout.write is able to print on the same line by replacing previous print
        if text_index == 0:
            self.start_time_global = time.time()
            print("Algorithm started")
        if text_index == 1:
            stdout.write("\rSolution will have a length of %d. Current length of amount of possibilities is %d" \
                % (self.length, len(str(self.total))))
        if text_index == 2:
            print("\n")
            if self.reverse == 1:
                print("Solution will be reversed")
            print("Starting length of real query : %d." % (len(str(self.query))))
            print("Real query :", self.query)
            print()
        if text_index == 3:
            length = len(str(self.query))
            left_space = self.limit-self.space
            percent = int((left_space*100)/self.limit)
            time_elapsed = int(time.time() - self.start_time_segment)
            stdout.write("\rCurrent length of real query %d. Cursor at position %d/%d (%d%%). Amount of \"7\" put %d. Time elapsed %ds." \
                % (length, left_space, self.limit, percent, self.seven_amount, time_elapsed))
        if text_index == 4:
            print("\n")
            print("Algorithm finished. Total time elapsed", time.time() - self.start_time_global)
            print("Solution have a length of", len(self.solution), "and have", self.solution.count("4"), "'4' and", \
                  self.solution.count("7"), "'7'")
            print()
    
    def is_lucky(self, number): #return 1 if number is a lucky number
        if any(x in str(number) for x in "01235689"):
            return 0
        return 1
    
    def get_palindrome_pair(self, length): #return amount of 4 and 7 that one side of a SLP can have
        pair = []
        is_odd = length % 2 #is_odd represent the possible middle digit contained in a odd length palindrome
        limit = length // 2 #we will simulate only one part of the palindrome because that's how the algorithm works
        total = 0
        for x in range(0, limit+1):
            if self.is_lucky(x*2) == 1 or self.is_lucky(x*2+is_odd) == 1 or \
            self.is_lucky(length-x*2) == 1 or self.is_lucky(length-x*2-is_odd) == 1:
                pair.append(x)
        return pair
    
    def get_possibilities(self, length): #determine how much possibility for a given length
        total = 0
        for i in self.pair:
            if i <= length:
                total += scipy.special.comb(length, i, exact=True) #exact=True return a int instead of a float
            else:
                break
        return total
    
    def get_lucky_from_index(self, index): #get lucky number from index. "4" is index 0, "7" is 1, "44" is 2 and so on
        binary = "{0:b}".format(index+2)   #I use binary numbers because 4 and 7 are like 0 and 1
        res = ""
        for idx, i in enumerate(binary[1:]):
            res += "4" if i == "0" else "7"
        return int(res)
    
    def exponential_search(self, length, target):
        idx = length-2 #start at end of array
        step = 1 #helps moving on the array 
        current = -1 #store current value
        best_idx = -1 #return idx
        best_val = 0 #return value
        over_it = False #once we've got over the target we will change the behavior of step
        
        while step > 0: #while we can move
            if over_it == True:
                step = step // 2 #need to reduce the steps to get precise

            current = self.get_possibilities(idx+1) #get the comb[idx] value
            if current <= target: # if true we update the best values because we're sure they are the current best
                if over_it == False: #exception when breaking the point for the first time
                    step = step // 4
                    over_it = True #we know we crossed the line so we now need to get precise
                    if idx == length-2: #another exception
                        step = 1
                best_val = current #saves the best values
                best_idx = idx
                idx += step
            else:
                idx -= step

            if over_it == False:
                step *= 2       #need to increase the step to go further and faster
        
        return best_idx+1, best_val
    
    def generate(self):
        self.space = self.limit
        self.solution = ["4"] * self.limit
        idx = 0
        remove = 1
        
        self.start_time_segment = time.time()
        while self.query > 0 and remove > 0:
            self.verbose_print(3)
            
            idx, remove = self.exponential_search(self.space, self.query)
            self.solution[idx] = "7"
            self.seven_amount += 1
            self.query -= remove
            self.space = idx
            for i in range(len(self.pair)): #update pair numbers
                self.pair[i] -= 1
            if self.pair[0] < 0: #delete because useless
                del self.pair[0]
        
        if self.query == 0:
            for i in range(self.pair[0]): #sometimes there is some remaining "7" to put
                self.solution[i] = "7"
                self.seven_amount += 1
        self.space = 0
        self.verbose_print(3)
        
        self.solution = "".join(self.solution)
    
    def get_solution(self, query): #let's go
        self.verbose_print(0)
        self.query = query
        index = 0
        while self.query > 0: #we need to know the length of the solution
            self.length = self.get_lucky_from_index(index) #we first get the length of out current lucky number
            self.pair = self.get_palindrome_pair(self.length) #we determine the pair numbers of the length
            self.limit = self.length // 2
            self.total = self.get_possibilities(self.limit) #we calculate how much possibilities exist
            self.query -= self.total #if query > 0 that means the solution is not in this length
            index += 1
            self.verbose_print(1)
        self.query += self.total - 1       #need to go back one step to know what is the real query
        
        self.reverse = False    #used to helps algorithm by a little bit
        if (self.query - self.total//2) > 0:
            self.query = self.total - self.query - 1
            self.reverse = True
        
        self.seven_amount = 0 #we assume that the solution is filled by 4 where some of 4s will be replaced by 7s
        self.verbose_print(2)
        
        self.generate() #everything
        
        if self.length%2 == 0: #we create the whole number from one side of it
            self.solution = self.solution[::-1] + self.solution
        else:                                        #and here we determine what is the value of the middle digit
            if self.is_lucky(self.seven_amount*2+1) == 1 or self.is_lucky((self.limit-self.seven_amount)*2) == 1:
                self.solution = self.solution[::-1] + "7" + self.solution
            else:
                self.solution = self.solution[::-1] + "4" + self.solution
        
        if self.reverse == True: #reverse the solution if needed
            self.solution = self.solution.replace("4","0")
            self.solution = self.solution.replace("7","4")
            self.solution = self.solution.replace("0","7")
        
        self.verbose_print(4)
        return self.solution

In [ ]:
import time

def print_v5(query):
    v5 = lucky_v5()
    start = time.time()
    sol = v5.get_solution(query)
    timeE = time.time() - start
    print("%19d | %f | %s (%d)" % (query, timeE, sol, len(sol)))

print("              query | time (s) | solution (length)")
for start, finish in zip([1, 240, 380000, 10**10, 10**15, 10**18, 10**100], [16, 245, 380005, 10**10+1, 10**15+1, 10**18+1, 10**100+1]):
    print()
    for query in range(start,finish):
        print_v5(query)

In [11]:
print(lucky_v5(True).get_solution(10**1000))

Algorithm started
Solution will have a length of 7444. Current length of amount of possibilities is 1089

Starting length of real query : 1000.
Real query : 999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999893645502559178915199154397671523804370734634138406542131914707648577192810749825145734723335764301995084169560305257259050163524861717437735261032432265053513311068310917879567372555058150940359592039566707727787394766659611374710490241745132114307122888555931588986266606877976899811237482742668930080

# Time comparison

Here you'll find a little script that will execute every algorithm and will display the amount of time each algorithm take to find the solution for different query.<br>
Queries are selected for a good comparison between each algorithm.

In [ ]:
import time

v1 = lucky_v1()
v2 = lucky_v2()
v3 = lucky_v3()
v4 = lucky_v4()
v5 = lucky_v5()
for i in [50, 250, 2500, 10**5, 10**6]:
    res = []
    print("computing query", i)
    for v, algo in enumerate([v1, v2, v3, v4, v5]):
        start = time.time()
        res.append(algo.get_solution(i))
        print("v"+str(v+1) + " in " + str(int((time.time()-start)*1000)) + " ms")
    if len(set(res)) == 1:
        print("Solutions verified OK")
    else:
        print("Difference in solutions has been found")
    print()

print("------------------\n")

for i in [10, 11, 44, 45, 114, 115]: #power of ten
    res = []
    print("computing query 10^"+str(i))
    for v, algo in enumerate([v3, v4, v5]):
        start = time.time()
        res.append(algo.get_solution(10**i))
        print("v"+str(v+3) + " in " + str(int((time.time()-start)*1000)) + " ms")
    if len(set(res)) == 1:
        print("Solutions verified OK")
    else:
        print("Difference in solutions has been found")
    print()